In [ ]:
from math import e, pi, cos, sin
import numpy as np
from random import *

m = 0.001  # масса шара
p = 0.07 # Дж/Тл
mu0 = 1.25663706 * 10 ** -6  # магнитная константа

S = 1 # мм^2

class Circuit:
    """U0 - напряжение конденсатора
       x0 - координата шара для запуска схемы
       C - емкость конденсатора
       L - индуктивность катушки
       n -плотность намотки
       x1 - начало катушки
       x2 - конец катушки
       R - дополнительное сопротивление схемы
       D - радиус катушки
       t0 - храним время зажигания системы. None -> цепь еще не включилась
       """

    def __init__(self, U0, x0, C, x1, x2, R, D, d):
        self.U0 = U0
        self.x0 = x0
        self.d = d
        self.n = 1 / d
        self.D = D
        self.L = mu0 * self.n**2 * (x2 - x1) * pi * D**2
        #print("Индуктивность схема:", self.L)
        self.C = C
        self.R = R
        self.x1 = x1
        self.x2 = x2
        self.t0 = -1
    def __str__(self):
        return str([self.U0, self.x0, self.C, self.x1, self.x2, self.R, self.D, self.d])

def main(circuits, n_circuits, v0):
    # складируем времена, координаты и скорости
    ts = []
    xs = []
    vs = []
    Is = []
    for i in range(n_circuits):
        Is.append([])
    # 2d array токов, iй схеме сопоставлен iй массив токов в каждый момент времени


    def current(t, vars, circuit):
        """Ток теперь функция координаты"""
        x = vars[0]
        if x >= circuit.x0 or circuit.t0 != -1:
            if circuit.t0 == -1:
                circuit.t0 = t # ОЧЕНЬ ВАЖНО ПОСЛЕ ВСЕГО ПОМЕНЯТЬ НА t
            gamma = circuit.R / (2 * circuit.L)
            omega0 = 1 / (circuit.L * circuit.C) ** 0.5
            if omega0 > gamma:
                #print("Живем в периодическом режиме")
                omega = (omega0 ** 2 - gamma ** 2) ** 0.5
                # print(circuit.C * circuit.U0 * e ** (-gamma * t) * (-gamma * np.cos(omega * (t - circuit.t0))))
                return circuit.C * circuit.U0 * e ** (-gamma * t) * (
                        -gamma * np.cos(omega * (t - circuit.t0)) + omega * np.sin(omega * (t - circuit.t0)))
            elif omega0 == gamma:
                return circuit.C * circuit.U0 * gamma ** 2 * (t - circuit.t0) * e ** (-gamma * (t - circuit.t0))
            else:
                #print("Живем в апериодическом режиме")
                epsilon = (- omega0 ** 2 + gamma ** 2) ** 0.5

                try:
                    return circuit.C * circuit.U0 * (epsilon ** 2 - gamma ** 2) / (2 * epsilon) * e**(-(t - circuit.t0) * gamma) * (
                                   e**((t - circuit.t0) * epsilon) - e**(-((t - circuit.t0) * epsilon)))
                except OverflowError:
                    return 0

        return 0



    def check_circuit(t, vars, circuit):
        x = vars[0]
        if x >= circuit.x0:
            if circuit.t0 == -1:
                circuit.t0 = t



    def force_by_induct(t, vars, circuit):
        I = current(t, vars, circuit)
        n = circuit.n
        length = circuit.x2 - circuit.x1
        D = circuit.D
        x = vars[0]
        x_new = x - (circuit.x2 + circuit.x1)/2  # координата шарика относительно катушки
        #print(I, n, length, D, x, x_new)

        return mu0 / 2 * I * n * 8 * D ** 2 * (
                1 / ((length + 2 * x_new) ** 2 + 4 * D ** 2) ** (3 / 2) - 1 / ((length - 2 * x_new) ** 2 + 4 * D ** 2) ** (
                    3 / 2)) * p / m


    def f(t, vars):
        x = vars[0]
        force = 0
        for circuit in circuits:
            check_circuit(t, vars, circuit)
            if circuit.t0 != -1:
                force += force_by_induct(t, vars, circuit)
        return np.array([vars[1], force])




    def step_handler(t, vars):
        """Обработчик шага"""
        ts.append(t)
        xs.append(vars[0])
        vs.append(vars[1])
        for i in range(n_circuits):
             Is[i].append(current(t, [vars[0], vars[1]], circuits[i]))


    # n_circuit
    #  circuits
    #circuits = [Circuit(U0=24, x0=-0.05, C=10 ** (-5), R=0.1, D=0.001, x1=-0.05, x2=0.05), ]
    #circuits = [Circuit(U0=24, x0=-0.05, C=10 ** (-3), R=0.0001, D=0.001, x1=-0.05, x2=0.05), ]
    # #n_circuit = 1
    # circuits = [Circuit(U0=24, x0=-0.05, C=10 ** (-3), R=0.0001, D=0.001, x1=-0.05, x2=0.05), Circuit(U0=24, x0=0.1, C=10 ** (-3), R=0.0001, D=0.001, x1=0.1, x2=0.2)]
    # n_circuit = 2



    def current_for_plot(t, vars, circuit):
        """тестовая функция чтобы было удобно плотить. поменяли только t на 0 чтобы работало"""
        x = vars[0]
        if x >= circuit.x0  or circuit.t0 != -1:
            if circuit.t0 == -1:
                circuit.t0 = 0 # ОЧЕНЬ ВАЖНО ПОСЛЕ ВСЕГО ПОМЕНЯТЬ НА t
            gamma = circuit.R / (2 * circuit.L)
            omega0 = 1 / (circuit.L * circuit.C)**0.5
            if omega0 > gamma:
                omega = 1 / (omega0**2 - gamma ** 2) ** 0.5
           # print(circuit.C * circuit.U0 * e ** (-gamma * t) * (-gamma * np.cos(omega * (t - circuit.t0))))
                return -circuit.C * circuit.U0 * e ** (-gamma * t) * (
                        -gamma * np.cos(omega * (t - circuit.t0)) + omega * np.sin(omega * (t - circuit.t0)))
            if omega0 == gamma:
                return circuit.C * circuit.U0 * gamma**2 * (t - circuit.t0) * e ** (-gamma * (t - circuit.t0))
            else:
                epsilon = (- omega0**2 + gamma ** 2) ** 0.5
                return circuit.C * circuit.U0 * (epsilon**2 - gamma**2)/(2*epsilon) * (e**(-(t - circuit.t0)*gamma)) * (e**((t - circuit.t0)*epsilon) - e**(-((t - circuit.t0)*epsilon)))


        return 0


#def main():
    """n_circuit - количество RLC цепей
        circuits- массив с параметрами RLC цепей"""
    # время движения
    tmax = 1
    import numpy as np
    from scipy.integrate import ode
    ODE = ode(f)  # тут есть f1 и f. f написано кривыми ручками и рассчитано на бумажке f1 посчитал вольфрам
    ODE.set_integrator('dopri5', max_step=0.001, nsteps=70000)
    ODE.set_solout(step_handler)
    # circuits = [Circuit(U0=24, x0=-0.05, C=10 ** (-3), R=0.0001, D=0.001, x1=-0.05, x2=0.05),
    #             Circuit(U0=24, x0=0.1, C=10 ** (-3), R=0.0001, D=0.001, x1=0.1, x2=0.2)]
    # n_circuit = 2

    ODE.set_initial_value(np.array([-0.05, v0]), 0)  # задание начальных значений
    ODE.integrate(tmax)  # решение ОДУ

    #print(xs)
    #print(vs)
    #print(ts)
    return ts, xs, vs, Is

    import matplotlib.pyplot as plt
    fig, axs = plt.subplots(3)
    fig.suptitle('Vertically stacked subplots')
    # axs[0].plot(ts[:700], xs[:700])
    # координата от времени
    axs[0].plot(ts, xs)
    #axs[0].set_xlim([0, 5])
    # axs[0].title("x(t)")
    # axs[1].plot(ts[1530:1550], vs[1530:1550])
    # скорость от времени
    axs[1].plot(ts, vs)
    # ток от времени
    axs[2].plot(ts, current(np.array(ts), [-0.05, 0], circuits[0]))
    #axs[1].set_xlim([0, 5])
    # axs[1].title("v(t)")

    x = np.arange(-2, 2, 0.01)
    # plt.plot(x, f1(0, [x, 0])[1])
    plt.show()
    #return ts, xs, vs, Is


All_possibilities = []
number_of_circuits = 4
N = 10**6
rho = 0.0175*10**(-6)
v_max = 0
kpd_max = 0
curcuits_max_kpd = []
circuits_max_vel = []
limits = [0.64258981, 11.71964023]
for i in range(N):
    circuits = []
    z = False
    all_x = []
    for tr in range(2*number_of_circuits):
        all_x.append(random())
    sorted(all_x)
    for j in range(number_of_circuits):
        u = 40*random()-20
        c = 10**(random()*8-6)
        R = 10**(random()*5-3)
        radius = np.sqrt(10**(-6)*choice([.5, .75, 1, 1.5, 2, 2.5 , 4, 6, 10, 16, 25, 35])/np.pi)
        width = (random()*19+1)/200
        x0 = random()*2-1
        x1 = all_x[j*2]
        x2 = all_x[j*2+1]
        R_sum = R + rho*(x2-x1)/(np.pi*radius**2)
        circuits.append(Circuit(U0=u, x0=x0, C=c, R=R_sum, D=width, x1=x1, x2=x2, d = 2*radius))
    v0 = 0
    A, B, C, D = main(circuits, number_of_circuits, )
    flag = False
    for l in range(number_of_circuits):
        Area = np.pi*(circuits[l].d/2)**2
        if max(abs(np.array(D[l])))>np.exp(limits[1])*Area**limits[0]:
            flag = True
            continue
    if flag:
        continue
    print(i, 'N')
    Energy = 0
    for circ in circuits:
        Energy+=(circ.C*circ.U0**2)/2
    if C[-1]>v_max:
        v_max = C[-1]
        circuits_max_vel = circuits
        print('------new velosity maximum--------')
        print(v_max)
        for circs in circuits:
            print(circs)
        print('------------------------')
        with open("/content/drive/MyDrive/gauss_gun_variation_collab_data/vmax_v2.txt", "a") as f:
            f.write(str(v_max) + "\n")
            for circs in circuits:
                f.write(str(circs) + "\n")
    if (0.001*(C[-1])**2/2)/(Energy)>kpd_max:
        kpd_max = (0.001*(C[-1])**2/2)/(Energy)
        circuits_max_kpd = circuits
        print('------new kpd maximum--------')
        print(kpd_max)
        for circs in circuits:
            print(circs)
        with open("/content/drive/MyDrive/gauss_gun_variation_collab_data/kpdmax_v2.txt", "a") as f:
            f.write(str(kpd_max) + "\n")
            for circs in circuits:
                f.write(str(circs) + "\n")
        print('------------------------')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/scipy/integrate/_ode.py:1177: UserWarning: dopri5: step size becomes too small
  self.messages.get(istate, unexpected_istate_msg)))


0 N
1 N
2 N
3 N


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:73: RuntimeWarning: overflow encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:73: RuntimeWarning: invalid value encountered in double_scalars


6 N
------new velosity maximum--------
3.2696243310154974e-10
[11.46707498210898, 0.6075986837675149, 11.296703730158788, 0.011741073663368295, 0.8648624168259923, 3.1860224523438894, 0.09016233371395777, 0.002763953195770684]
[-10.690887960089785, 0.3981001093237453, 0.00013607357190016355, 0.11502304547119357, 0.8282104079246669, 0.01588418981867115, 0.07519955575844177, 0.0011283791670955124]
[14.754278075465791, -0.739105547275837, 3.0526060364019977e-06, 0.9084960291487864, 0.99604323588228, 3.4826652691622417, 0.03371119622343717, 0.0015957691216057306]
[15.668083255674986, -0.04608371951928403, 0.031868242715997135, 0.9828046078403605, 0.9100779935072418, 0.0010542431518411404, 0.068688306393892, 0.006675581178124545]
------------------------
------new kpd maximum--------
7.159005344079341e-26
[11.46707498210898, 0.6075986837675149, 11.296703730158788, 0.011741073663368295, 0.8648624168259923, 3.1860224523438894, 0.09016233371395777, 0.002763953195770684]
[-10.690887960089785, 0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:100: RuntimeWarning: invalid value encountered in double_scalars


...

54 N
55 N
56 N
57 N
58 N
59 N
------new velosity maximum--------
0.03647093918773203
[2.5483273159541433, 0.7592310539891836, 0.815776584732596, 0.7728279200315598, 0.1433014468690259, 0.0037090146885974773, 0.00997895419853435, 0.005641895835477563]
[10.662695763164791, -0.8080451287955495, 0.048336671181476734, 0.0013552186755946272, 0.7834062363233494, 0.008480429931461272, 0.08621617751717013, 0.00451351666838205]
[-3.4451956031378295, 0.9522862696589336, 2.5465759834922506, 0.36077973843309075, 0.37701505147885794, 0.006019649007060807, 0.03817254622327896, 0.002763953195770684]
[-17.517525750450336, -0.6229177446984937, 2.482752142696271e-05, 0.04360968061991155, 0.7161026877238098, 1.8728241465483124, 0.06904055557529451, 0.0007978845608028653]
------------------------
------new kpd maximum--------
3.2420787883941215e-08
[2.5483273159541433, 0.7592310539891836, 0.815776584732596, 0.7728279200315598, 0.1433014468690259, 0.0037090146885974773, 0.00997895419853435, 0.00564189583547

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:100: RuntimeWarning: overflow encountered in double_scalars


478 N
479 N
480 N
481 N
482 N
483 N
484 N
485 N
486 N
487 N
488 N
489 N
490 N
491 N
492 N
493 N
494 N
495 N
496 N
497 N
498 N
501 N
502 N
503 N
504 N
505 N
506 N
507 N
508 N
509 N
510 N
511 N
512 N
513 N
514 N
515 N
516 N
517 N
518 N
519 N
520 N
521 N
522 N
525 N
526 N
527 N
528 N
529 N
530 N
531 N
532 N
533 N
534 N
535 N
537 N
539 N
542 N
543 N
545 N
546 N
547 N
548 N
549 N
550 N
551 N
552 N
553 N
554 N
555 N
556 N
557 N
558 N
559 N
560 N
562 N
563 N
564 N
566 N
567 N
568 N
571 N
572 N
573 N
574 N
575 N
576 N
577 N
578 N
579 N
580 N
581 N
582 N
583 N
584 N
586 N
588 N
589 N
590 N
591 N
592 N
593 N
594 N
595 N
597 N
598 N
599 N
600 N
601 N
602 N
603 N
604 N
605 N
606 N
607 N
608 N
609 N
611 N
612 N
613 N
614 N
615 N
616 N
617 N
618 N
619 N
621 N
623 N
624 N
626 N
627 N
628 N
629 N
631 N
632 N
633 N
634 N
635 N
636 N
638 N
639 N
640 N
641 N
643 N
644 N
645 N
646 N
647 N
648 N
650 N
651 N
652 N
653 N
654 N
656 N
658 N
659 N
661 N
663 N
664 N
------new velosity maximum--------
0.521849995

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:109: RuntimeWarning: invalid value encountered in double_scalars


Выходные данные были обрезаны до нескольких последних строк (5000).
13251 N
13252 N
13253 N
13254 N
13255 N
13256 N
13257 N
13259 N
13261 N
13262 N
13263 N
13264 N
13265 N
13266 N
13267 N
13268 N
13269 N
13270 N
13272 N
13273 N
13274 N
13275 N
13276 N
13277 N
13278 N
13279 N
13280 N
13281 N
13282 N
13283 N
13284 N
13285 N
13286 N
13287 N
13288 N
13289 N
13290 N
13292 N
13293 N
13294 N
13295 N
13296 N
13297 N
13298 N
13300 N
13301 N
13302 N
13303 N
13304 N
13305 N
13306 N
13307 N
13308 N
13310 N
13312 N
13313 N
13315 N
13316 N
13317 N
13318 N
13319 N
13320 N
13321 N
13323 N
13324 N
13326 N
13328 N
13329 N
13330 N
13331 N
13332 N
13333 N
13334 N
13335 N
13336 N
13337 N
13338 N
13339 N
13340 N
13341 N
13342 N
13343 N
13344 N
13345 N
13346 N
13347 N
13348 N
13349 N
13351 N
13352 N
13355 N
13356 N
13357 N
13358 N
13359 N
13360 N
13362 N
13363 N
13364 N
13365 N
13366 N
13367 N
13368 N
13369 N
13370 N
13371 N
13372 N
13373 N
13374 N
13375 N
13378 N
13379 N
13380 N
13381 N
13382 N
13384 N
1338